In [1]:
from pyHalo.Halos.HaloModels.TNFW import TNFWSubhalo
from pyHalo.Halos.lens_cosmo import LensCosmo
import numpy as np

### Simulate emulator output 

In [2]:
# here we'll just pretend we have some information output by the emulator
infall_mass = 10 ** 8
infall_concentration = 16
final_bound_mass = 5 * 10**7
x_coordinate = 20 # projected x position [kpc]
y_coordinate = 10 # projected y position [kpc]

# this should be the redshift at the time of lensing, which should also be the time when the final bound mass is evaluated
redshift = 0.5 
# we also need the source redshift for a conversion into angular units; 
# this can eventually be a user-specified arguement, for now just assume z_source = 2
source_redshift = 2.0


# need to create an instance of a LensCosmo class
lens_cosmo = LensCosmo(redshift, source_redshift)


### Halo density profile

We need to stick with halo density profiles for which we can analytically compute the deflection angles. The profile implemented in pyHalo has the following form

\begin{equation}
\rho\left(r\right) = \rho_s \frac{1}{x \left(1+x\right)^2} \frac{\tau^2}{\tau^2 + x^2}
\end{equation}

where $\rho_s$ is the characteristic central density, $x \equiv r / r_s$, and $\tau = r_t / r_s$. Here, $r_s$ is the halo scale radius, and $r_t$ is effectively a truncation radius where the logarithmic slope transitions to $-5$. If we know the final bound mass, the concentration, and the mass at infall, we can solve for $r_t$, which is done in the following cell. 

In [3]:
from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator


def tnfw_mass_fraction(tau, c):
    """
    This function returns the fraction = final_mass/initial_mass, assuming a truncated NFW profile
    :param tau: the truncation radius in units of the scale radius 
    :param c: the halo concentration
    """
    x = c
    Rs = 1.0
    r_trunc = tau * Rs
    func = (r_trunc ** 2 * (-2 * x * (1 + r_trunc ** 2) + 4 * (1 + x) * r_trunc * np.arctan(x / r_trunc) -
                            2 * (1 + x) * (-1 + r_trunc ** 2) * np.log(Rs) + 2 * (1 + x) * (-1 + r_trunc ** 2) * np.log(Rs * (1 + x)) +
                            2 * (1 + x) * (-1 + r_trunc ** 2) * np.log(Rs * r_trunc) -
                            (1 + x) * (-1 + r_trunc ** 2) * np.log(Rs ** 2 * (x ** 2 + r_trunc ** 2)))) / (2. * (1 + x) * (1 + r_trunc ** 2) ** 2)
    mass_loss = func / (np.log(1+c)-c/(1+c))
    return mass_loss
    
def tau_mf_interpolator():

    N = 100
    tau = np.logspace(-2, 3, N)
    concentration = np.linspace(1.1, 100, N)
    # mass_fraction_1d = np.logspace(-1.45, -0.02, N)
    mass_fraction_1d = np.logspace(-2, -0.01, N)
    log10tau_2d = np.zeros((N, N))

    # This computes the value of tau that correponds to each pair of (concentration, mass_loss) 
    for i, con_i in enumerate(concentration):
        log10final_mass = np.log10(tnfw_mass_fraction(tau, con_i))
        mfinterp = interp1d(log10final_mass, np.log10(tau))
        for j, mass_j in enumerate(mass_fraction_1d):
            log10t = mfinterp(np.log10(mass_j))
            log10tau_2d[i,j] = log10t

    interp_points = (np.log10(concentration), np.log10(mass_fraction_1d))
    interpolator = RegularGridInterpolator(interp_points,log10tau_2d, bounds_error=False)
    return interpolator

# we will call this in the new subhalo class
truncation_radius_interpolator = tau_mf_interpolator()
# EXAMPLE:
c = 15
log10_c = np.log10(c)
infall_mass = 10 ** 8
final_bound_mass = 2.0 * 10**7
log10_fraction = np.log10(final_bound_mass/infall_mass)
point = (log10_c, log10_fraction)
log10_tau = truncation_radius_interpolator(point)
print(log10_tau)
print('r_t over r_s: ', 10 ** log10_tau)


0.11738025168567481
r_t over r_s:  1.310328694724475


### This is the class we will eventually put into pyHalo (togetherr with the interpolation function above)

In [4]:
from pyHalo.Halos.halo_base import Halo

class TNFWSubhaloEmulator(Halo):
    """
    Defines a truncated NFW halo that is a subhalo of the host dark matter halo
    """
    def __init__(self, infall_mass, x, y, final_bound_mass, infall_concentration, redshift, 
                 lens_cosmo_instance, unique_tag=None):
        """
        See documentation in base class (Halos/halo_base.py)
        """
        self._lens_cosmo = lens_cosmo_instance
        
        r3d = None
        profile_definition = 'TNFW'
        sub_flag = True
        args = None
        if unique_tag is None:
            unique_tag = np.random.rand()
        
        # set the concentration
        self.c = infall_concentration
        self._bound_mass_fraction = final_bound_mass/infall_mass
        x_arcsec = x / self._lens_cosmo.cosmo.kpc_proper_per_asec(redshift)
        y_arcsec = y / self._lens_cosmo.cosmo.kpc_proper_per_asec(redshift)
        super(TNFWSubhaloEmulator, self).__init__(infall_mass, x_arcsec, y_arcsec, r3d, profile_definition, redshift, sub_flag,
                                           lens_cosmo_instance, args, unique_tag)
        
    @property
    def lenstronomy_params(self):
        """
        See documentation in base class (Halos/halo_base.py)
        """
        if not hasattr(self, '_kwargs_lenstronomy'):

            [concentration, rt] = self.profile_args
            Rs_angle, theta_Rs = self._lens_cosmo.nfw_physical2angle(self.mass, concentration, self.z)

            x, y = np.round(self.x, 4), np.round(self.y, 4)

            Rs_angle = np.round(Rs_angle, 10)
            theta_Rs = np.round(theta_Rs, 10)
            r_trunc_arcsec = rt / self._lens_cosmo.cosmo.kpc_proper_per_asec(self.z)

            kwargs = [{'alpha_Rs': self._rescale_norm * theta_Rs, 'Rs': Rs_angle,
                      'center_x': x, 'center_y': y, 'r_trunc': r_trunc_arcsec}]

            self._kwargs_lenstronomy = kwargs

        return self._kwargs_lenstronomy, None
        
    @property
    def profile_args(self):
        """
        See documentation in base class (Halos/halo_base.py)
        """
        if not hasattr(self, '_profile_args'):
            point = (np.log10(self.c), np.log10(self._bound_mass_fraction))
            truncation_radius_kpc = truncation_radius_interpolator(point)

            self._profile_args = (self.c, float(truncation_radius_kpc))

        return self._profile_args

    @property
    def params_physical(self):
        """
        See documentation in base class (Halos/halo_base.py)
        """
        if not hasattr(self, '_params_physical'):
            [concentration, rt] = self.profile_args
            rhos, rs, r200 = self._lens_cosmo.NFW_params_physical(self.mass, concentration, self.z)
            self._params_physical = {'rhos': rhos, 'rs': rs, 'r200': r200, 'r_trunc_kpc': rt}

        return self._params_physical
    
    @property
    def lenstronomy_ID(self):
        """
        See documentation in base class (Halos/halo_base.py)
        """

        return ['TNFW']

In [5]:
# single halo
truncated_nfw_profile = TNFWSubhaloEmulator(infall_mass, x_coordinate, y_coordinate, 
                                            final_bound_mass, infall_concentration, redshift, lens_cosmo)
keyword_arguments_lenstronomy, _ = truncated_nfw_profile.lenstronomy_params
print('keyword arguements: ', keyword_arguments_lenstronomy)
print('lens model: ', truncated_nfw_profile.lenstronomy_ID)

keyword arguements:  [{'alpha_Rs': 0.0007951274, 'Rs': 0.0808691769, 'center_x': 3.1693, 'center_y': 1.5847, 'r_trunc': 0.02081310492864744}]
lens model:  ['TNFW']


### create a full population of halos

In [6]:
from pyHalo.single_realization import Realization
from pyHalo.defaults import set_default_kwargs

masses = 10**np.random.randint(6, 10, 100)
xpositions_kpc = 30*np.random.randn(100)
ypositions_kpc = 30*np.random.randn(100)
final_bound_masses = np.absolute(np.random.randn(100)) * masses
redshifts = [redshift] * len(masses)
infall_concentrations = np.random.uniform(8, 25, size=len(masses))
halo_list = []
for i in range(0, len(masses)):
    halo = TNFWSubhaloEmulator(masses[i], xpositions_kpc[i], ypositions_kpc[i], final_bound_masses[i], infall_concentrations[i],
                              redshifts[i], lens_cosmo)
    halo_list.append(halo)

minimum_mass_emulator = 10 ** 6
maximum_mass_emulator = 10 ** 10
kwargs_setup = {'log_mlow': np.log10(minimum_mass_emulator),
               'log_mhigh': np.log10(maximum_mass_emulator)}
prof_params = set_default_kwargs(kwargs_setup, source_redshift)
realization = Realization.from_halos(halo_list, lens_cosmo, prof_params, msheet_correction=False, rendering_classes=None)
    

In [7]:
print(realization.lensing_quantities())

(['TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW', 'TNFW'], [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0